In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import set_seed
import torchaudio.transforms as T
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ----------------------------
# 配置
# ----------------------------
set_seed(42)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SAMPLE_RATE = 16000
N_MELS = 64
MAX_TIME = 300        # 时间步上限（约 9.6 秒）
BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 1e-3

# 情感标签（与 TwinkStart/RAVDESS 一致）
EMOTIONS = ["neutral", "calm", "happy", "sad", "angry", "fearful", "disgust", "surprised"]

In [ ]:
# ----------------------------
# 1. 加载数据集
# ----------------------------
print("📥 Loading RAVDESS from Hugging Face...")
dataset = load_dataset("TwinkStart/RAVDESS")

# 划分训练/测试（原数据集无验证集，我们从 train 分出 val）
train_val = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_ds = train_val["train"]
val_ds = train_val["test"]
test_ds = dataset["test"]

print(f"Train: {len(train_ds)}, Val: {len(val_ds)}, Test: {len(test_ds)}")

In [ ]:
# ----------------------------
# 2. 特征提取函数
# ----------------------------
def extract_log_mel(audio_array, orig_sr):
    # 转为 tensor
    waveform = torch.tensor(audio_array).float()
    if waveform.ndim == 1:
        waveform = waveform.unsqueeze(0)  # (1, T)

    # 重采样
    if orig_sr != SAMPLE_RATE:
        resampler = T.Resample(orig_freq=orig_sr, new_freq=SAMPLE_RATE)
        waveform = resampler(waveform)

    # 转单声道
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Log-Mel Spectrogram
    mel_spec = T.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=N_MELS
    )(waveform)
    log_mel = T.AmplitudeToDB()(mel_spec)  # (1, n_mels, time)

    # 时间维度对齐
    time_dim = log_mel.shape[2]
    if time_dim > MAX_TIME:
        log_mel = log_mel[:, :, :MAX_TIME]
    else:
        pad = torch.zeros(1, N_MELS, MAX_TIME - time_dim)
        log_mel = torch.cat([log_mel, pad], dim=2)

    return log_mel

In [ ]:
# ----------------------------
# 3. 自定义 collate_fn（用于 DataLoader）
# ----------------------------
def collate_fn(batch):
    log_mels = []
    labels = []
    for item in batch:
        audio = item["audio"]
        label = item["label"]
        log_mel = extract_log_mel(audio["array"], audio["sampling_rate"])
        log_mels.append(log_mel)
        labels.append(label)
    return torch.stack(log_mels), torch.tensor(labels, dtype=torch.long)

In [ ]:
# ----------------------------
# 4. 构建 DataLoader
# ----------------------------
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=2)

In [ ]:
# ----------------------------
# 5. CNN 模型
# ----------------------------
class AudioEmotionCNN(nn.Module):
    def __init__(self, num_classes=8, n_mels=64, max_time=300):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2)
        self.dropout = nn.Dropout(0.3)

        h = n_mels // 4
        w = max_time // 4
        self.fc = nn.Linear(64 * h * w, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)

In [ ]:
# ----------------------------
# 6. 训练函数
# ----------------------------
def train():
    model = AudioEmotionCNN(num_classes=8, n_mels=N_MELS, max_time=MAX_TIME).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    best_val_acc = 0.0

    for epoch in range(EPOCHS):
        # Training
        model.train()
        train_loss = 0.0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                logits = model(x)
                preds = logits.argmax(dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(y.cpu().numpy())

        val_acc = accuracy_score(val_labels, val_preds)
        print(f"Train Loss: {train_loss/len(train_loader):.4f}, Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")

    # Final test
    print("\n🧪 Testing best model...")
    model.load_state_dict(torch.load("best_model.pth", map_location=DEVICE))
    model.eval()
    test_preds, test_labels = [], []
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            logits = model(x)
            preds = logits.argmax(dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(y.cpu().numpy())

    test_acc = accuracy_score(test_labels, test_preds)
    print(f"\n✅ Final Test Accuracy: {test_acc:.4f}")
    print("\n📊 Classification Report:")
    print(classification_report(test_labels, test_preds, target_names=EMOTIONS, digits=4))

In [ ]:

if __name__ == "__main__":
    train()